In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import time
from datetime import date
import calendar
from config import key
import json
import pprint
import grequests

C:\Users\dines\Anaconda3\lib\site-packages\grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (C:\\Users\\dines\\Anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)', 'urllib3.util (C:\\Users\\dines\\Anaconda3\\lib\\site-packages\\urllib3\\util\\__init__.py)']. 
  curious_george.patch_all(thread=False, select=False)
C:\Users\dines\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
music_file = "music_data.csv"
music_data_df = pd.read_csv(music_file)
music_data_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,1/1/2017,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,1/1/2017,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,1/1/2017,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/1/2017,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,1/1/2017,ec


In [3]:
music_data_df = music_data_df.rename(columns = {'Region':'country_codes'})
music_data_df['country_codes'] = music_data_df['country_codes'].str.upper()
music_data_df.head()

,Position,Track Name,Artist,Streams,URL,Date,country_codes
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,1/1/2017,EC
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,1/1/2017,EC
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,1/1/2017,EC
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/1/2017,EC
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,1/1/2017,EC


In [4]:
tracks_df = music_data_df[['Track Name', 'Artist']].groupby(['Track Name', 'Artist']).nunique()
tracks_df = tracks_df[["Track Name","Artist"]].copy()
tracks_df.rename(columns={"Track Name":"track_count","Artist":"art_count"}, inplace=True)
tracks_df = tracks_df.reset_index()
tracks_df.rename(columns={"Track Name":"Track"}, inplace=True)
tracks_df_unique = tracks_df[["Track","Artist"]].copy()
tracks_df_unique.head()

,Track,Artist
0,"""All That Is or Ever Was or Ever Will Be""",Alan Silvestri
1,"""Read All About It, Pt. III""",Emeli Sandé
2,#99,JVG
3,#Askip,Black M
4,#Biziz - feat. Lil Bege,Reynmen


In [5]:
for index,row in tracks_df_unique.iterrows(): 
    track = row.Track.replace("#","%23")
    artist = row.Artist.replace("#","%23")
    url =  f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={key}&artist={artist}&track={track}&format=json"
    tracks_df_unique.at[index,"get_url"]= url

tracks_df_unique_100 = tracks_df_unique["get_url"].copy()
tracks_df_unique_100

0        http://ws.audioscrobbler.com/2.0/?method=track...
1        http://ws.audioscrobbler.com/2.0/?method=track...
2        http://ws.audioscrobbler.com/2.0/?method=track...
3        http://ws.audioscrobbler.com/2.0/?method=track...
4        http://ws.audioscrobbler.com/2.0/?method=track...
5        http://ws.audioscrobbler.com/2.0/?method=track...
6        http://ws.audioscrobbler.com/2.0/?method=track...
7        http://ws.audioscrobbler.com/2.0/?method=track...
8        http://ws.audioscrobbler.com/2.0/?method=track...
9        http://ws.audioscrobbler.com/2.0/?method=track...
10       http://ws.audioscrobbler.com/2.0/?method=track...
11       http://ws.audioscrobbler.com/2.0/?method=track...
12       http://ws.audioscrobbler.com/2.0/?method=track...
13       http://ws.audioscrobbler.com/2.0/?method=track...
14       http://ws.audioscrobbler.com/2.0/?method=track...
15       http://ws.audioscrobbler.com/2.0/?method=track...
16       http://ws.audioscrobbler.com/2.0/?method=track.

In [28]:
# A list to hold our things to do via async
async_list = []

for u in tracks_df_unique_100.head(100):
    action_item = grequests.get(u, timeout=10)
    async_list.append(action_item)

response = grequests.map(async_list)


In [29]:

response = [x for x in response if x is not None ]
len(response)

79

In [8]:
"""
for index,row in tracks_df_unique.head(1000).iterrows():
    try:  
        track = row.Track.replace("#","%23")
        artist = row.Artist.replace("#","%23")
        url =  f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={key}&artist={artist}&track={track}&format=json"
        response = requests.get(url).json()
        tracks_df_unique.at[index,"album"] = response["track"]["album"]["title"]
        tracks_df_unique.at[index,"genre"] = response["track"]["toptags"]["tag"][0]["name"]
        
    except (AttributeError,KeyError, IndexError):
        x=1

tracks_df_unique.head()
"""

'\nfor index,row in tracks_df_unique.head(1000).iterrows():\n    try:  \n        track = row.Track.replace("#","%23")\n        artist = row.Artist.replace("#","%23")\n        url =  f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={key}&artist={artist}&track={track}&format=json"\n        response = requests.get(url).json()\n        tracks_df_unique.at[index,"album"] = response["track"]["album"]["title"]\n        tracks_df_unique.at[index,"genre"] = response["track"]["toptags"]["tag"][0]["name"]\n        \n    except (AttributeError,KeyError, IndexError):\n        x=1\n\ntracks_df_unique.head()\n'

In [9]:
tracks_df_unique.head()

,Track,Artist,get_url
0,"""All That Is or Ever Was or Ever Will Be""",Alan Silvestri,http://ws.audioscrobbler.com/2.0/?method=track...
1,"""Read All About It, Pt. III""",Emeli Sandé,http://ws.audioscrobbler.com/2.0/?method=track...
2,#99,JVG,http://ws.audioscrobbler.com/2.0/?method=track...
3,#Askip,Black M,http://ws.audioscrobbler.com/2.0/?method=track...
4,#Biziz - feat. Lil Bege,Reynmen,http://ws.audioscrobbler.com/2.0/?method=track...


In [10]:
additional_info = []
row={}

for x in response:
    row ={}
    try:
        row["track"] = x.json()["track"]["name"]
        row["album"] = x.json()["track"]["album"]["title"]
        row["artist"] = x.json()["track"]["artist"]["name"]
        row["genre"] = x.json()["track"]["toptags"]["tag"][0]["name"]
        
        additional_info.append(row)
    except (IndexError,KeyError):
        a=1
additional_info = pd.DataFrame(additional_info)

In [11]:
additional_info

,album,artist,genre,track
0,#SELFIE,The Chainsmokers,electronic,#SELFIE
1,Who Dat Boy / 911,"Tyler, the Creator",rap,911 / Mr. Lonely
2,A Christmas Carol,Alan Silvestri,christmas,A Christmas Carol Main Title
3,Alaska,Maggie Rogers,electronic,Alaska
4,Fronte Del Palco,Vasco Rossi,italian,Albachiara
5,Sellaista Elämä On,Juha Vainio,1980s,Albatrossi
6,Amárrame,Mon Laferte,Juanes,Amárrame
7,Anatolia,Pentagram,anadolu rock,Anatolia
8,Anthrazit,RAF Camora,Hip-Hop,Andere Liga
9,Christmas,Michael Bublé,christmas,Ave Maria
